In [10]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten
from sklearn.model_selection import train_test_split

In [11]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [12]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONSPEED-SML-RSIDESTEPS-V0.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motionspeed-sml-rsidesteps-2.h5'

In [13]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, [3, 7]]
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, [3, 7]]

ALL_X_COMBINED_DATA = np.concatenate((ALL_X_TRAIN_DATA, ALL_X_TEST_DATA))
ALL_Y_COMBINED_DATA = np.concatenate((ALL_Y_TRAIN_DATA -1 , ALL_Y_TEST_DATA -1))

ALL_X_TRAIN_DATA, ALL_X_TEST_DATA, ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA = train_test_split(ALL_X_COMBINED_DATA, ALL_Y_COMBINED_DATA, test_size=0.2, random_state=42)

print(ALL_Y_TRAIN_DATA[:100])

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

[0 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 1 0 0
 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0]
(32175, 2)
(32175,)
(8044, 2)
(8044,)


In [14]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [15]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [16]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 3 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 3 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 3 else len(CLASSICATIONS)

In [17]:
model2_motion = Sequential()
model2_motion.add(InputLayer((2)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 8)                 24        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 105
Trainable params: 105
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12


1006/1006 [==============================] - 2s 2ms/step - loss: 0.5913 - acc: 0.6985 - val_loss: 0.5571 - val_acc: 0.6353
Epoch 2/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.5358 - acc: 0.6979 - val_loss: 0.5410 - val_acc: 0.6062
Epoch 3/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.5292 - acc: 0.6873 - val_loss: 0.5293 - val_acc: 0.6451
Epoch 4/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.5277 - acc: 0.6746 - val_loss: 0.5359 - val_acc: 0.6095
Epoch 5/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.5277 - acc: 0.6746 - val_loss: 0.5273 - val_acc: 0.7176
Epoch 6/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.5229 - acc: 0.6894 - val_loss: 0.5102 - val_acc: 0.7560
Epoch 7/12
1006/1006 [==============================] - 1s 1ms/step - loss: 0.4960 - acc: 0.7642 - val_loss: 0.4799 - val_acc: 0.8066
Epoch 8/12
1006/1006 [==============================] - 1s 1ms/step - los